In [4]:
import cv2
import mediapipe as mp
import time
import math

# Initialize MediaPipe pose class and drawing class
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize pose estimation
pose = mp_pose.Pose()

# Replace this with your video file path
video_path = r"4367576-hd_1920_1080_30fps.mp4"
cap = cv2.VideoCapture(video_path)

# Variables to track push-up count and timing
pushup_count = 0
start_time = None
in_down_position = False

def calculate_angle(a, b, c):
    """Calculate angle between three points: shoulder (a), elbow (b), wrist (c)"""
    ang = math.degrees(
        math.atan2(c.y - b.y, c.x - b.x) - math.atan2(a.y - b.y, a.x - b.x)
    )
    return abs(ang)

# Get video properties for saving output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create VideoWriter object
output_path = 'pushup_output_4367576-hd_1920_1080_30fps.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for natural (selfie-view) visualization
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and find pose
    results = pose.process(rgb_frame)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        landmarks = results.pose_landmarks.landmark

        # Get relevant landmarks for arm angle
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

        # Calculate arm angle
        arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

        # Detect push-up down position
        if arm_angle < 50:
            if not in_down_position:
                in_down_position = True
        elif arm_angle > 190:
            if in_down_position:
                pushup_count += 1
                in_down_position = False
                if start_time is None:
                    start_time = time.time()

        # Calculate duration
        duration = time.time() - start_time if start_time else 0

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2.0  # Bigger text
        thickness = 4     # Bolder text

        cv2.putText(frame, f'Push-ups: {pushup_count}', (10, 60), font, font_scale, (0, 255, 0), thickness)
        cv2.putText(frame, f'Arm Angle: {int(arm_angle)} deg', (10, 130), font, font_scale, (255, 0, 0), thickness)
        cv2.putText(frame, f'Duration: {int(duration)}s', (10, 200), font, font_scale, (255, 255, 0), thickness)
        # Write the frame to output video
        out.write(frame)
        



    # Resize frame to medium size
    height, width = frame.shape[:2]
    new_width = 1240
    new_height = int((new_width / width) * height)
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Show the resized frame
    cv2.imshow('Push-up Counter', resized_frame)


    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()

cv2.destroyAllWindows()



In [30]:
import os

import requests
 
# URL to download Tesseract from

url = "https://videos.pexels.com/video-files/4367576/4367576-hd_1920_1080_30fps.mp4"
 
# Get the file name from the URL

file_name = url.split("/")[-1]
 
# Download the file

response = requests.get(url)
 
# Check if the file was downloaded successfully

if response.status_code == 200:

    # Save the file to the current directory

    with open(file_name, "wb") as f:

        f.write(response.content)

    print("Tesseract file downloaded and saved as", file_name)

else:

    print("Error downloading the Tesseract file")
 

Tesseract file downloaded and saved as 4367576-hd_1920_1080_30fps.mp4
